In [16]:
%matplotlib inline
import tensorflow as tf
import matplotlib as plt
import glob
import numpy as np
from PIL import Image
import mltools as ml

trying to use the example from 
[http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)

denote number of data  
mesa: 10015  
forest: 10047   
desert: 10142  
jungle size:  10238   
eh size:  9261  
training size: 39762  
test size: 9941  
total size:  49703

In [17]:
labels ={"mesa":0, "forest":1, "desert":2,"jungle":3,"eh":4}
tfrecords_filename = '/notebooks/Minecraft-AI/mc-data/mesa_forest_desert_jungle_train.tfrecords'
tfrecords_test_filename='/notebooks/Minecraft-AI/mc-data/mesa_forest_desert_jungle_test.tfrecords'

In [18]:
mesa_files = glob.glob( '/notebooks/Minecraft-AI/mc-data/mesa_splited/**/*.jpg')
forest_files = glob.glob('/notebooks/Minecraft-AI/mc-data/forest_splited/**/*.jpg')
desert_files = glob.glob('/notebooks/Minecraft-AI/mc-data/desert_splited/**/*.jpg')
jungle_files = glob.glob('/notebooks/Minecraft-AI/mc-data/jungle_splited/**/*.jpg')
# eh_files= glob.glob('/notebooks/Minecraft-AI/mc-data/eh_splited/**/*.jpg')
mesa_pairs = zip(mesa_files, "0"*len(mesa_files))
forest_pairs = zip(forest_files, "1"*len(forest_files))
desert_pairs = zip(desert_files, "2"*len(desert_files))
jungle_pairs = zip(jungle_files, "3"*len(jungle_files))
eh_pairs = zip(eh_files, "4"*len(eh_files))

In [19]:
print "mesa size: ", len(mesa_pairs)
print "forest size: ", len(forest_pairs)
print "desert size: ", len(desert_pairs)
print "jungle size: ", len(jungle_pairs)
print "eh size: ", len(eh_pairs)

mesa size:  10015
forest size:  10047
desert size:  10142
jungle size:  10238
eh size:  9261


In [20]:
filename_pairs = mesa_pairs+forest_pairs+desert_pairs+jungle_pairs#+eh_pairs

In [21]:
np.random.shuffle(filename_pairs)
portion=int(0.8*len(filename_pairs))
train_pairs = filename_pairs[:portion]
test_pairs = filename_pairs[portion:]

In [22]:
print train_pairs[:5]

[('/notebooks/Minecraft-AI/mc-data/forest_splited/006/forest_3389.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/desert_splited/009/desert_8755.jpg', '2'), ('/notebooks/Minecraft-AI/mc-data/mesa_splited/003/mesa_2938.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/jungle_splited/005/jungle_5040.jpg', '3'), ('/notebooks/Minecraft-AI/mc-data/forest_splited/001/forest_10893.jpg', '1')]


In [23]:
print "training size:", len(train_pairs)
print "test size:", len(test_pairs)
print "total size: ", len(filename_pairs)

training size: 32353
test size: 8089
total size:  40442


In [24]:

train_writer = tf.python_io.TFRecordWriter(tfrecords_filename)
test_writer = tf.python_io.TFRecordWriter(tfrecords_test_filename)

In [25]:
# helper functions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [26]:
# Let's collect the real images to later on compare
# to the reconstructed ones
# original_images = []

In [27]:
def write_in_tfrecord(filename_pairs, writer):
    for img_path, label in filename_pairs:

        img = np.array(Image.open(img_path))
        
        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        # convert to 1d array and convert to respective
        # shape that image used to have.
        height = img.shape[0]
        width = img.shape[1]

        # Put in the original images into array
        # Just for future check for correctness
#         original_images.append((img,label))

        img_raw = img.tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'image_raw': _bytes_feature(img_raw),
            'label': _int64_feature(int(label))}))

        writer.write(example.SerializeToString())

In [28]:
write_in_tfrecord(train_pairs,train_writer)

In [29]:
write_in_tfrecord(test_pairs,test_writer)

In [30]:
train_writer.close()
test_writer.close()